In [1]:
%matplotlib inline
from numpy import *
import matplotlib.pyplot as plt
from matplotlib import cm
import pandas as pd
from numpy import random
from numba import jit,njit,prange
from scipy import fftpack 

mov_dir = 'mov/2D/' # Directory for the snapshots 

# CHOICE OF PARAMETERS
N = 1024 # box size
L = 1024 # physical size in units of L_1=1/(R_1H_1)
delta=1.0/3.0 # Courant parameter, same as Gorghetto et al.
#delta=0.1
dx=L/N # space step
dt = delta*dx # time step (comoving)
t = 1.0*dt # initial time
tmax = int(0.5*N*dx/dt) # light crossing time (do not evolve beyond this timestep)

alpha = 1 # For the PRS fudge factor, keeping the strings fixed width in comoving coords.
era = 1 # 2 for matter domination, 1 for radiation domination (in PRS trick)
lambdaPRS = 0.25 #Quartic coupling scaled by PRS trick
fa = 1 # In natural fa units 
ms = sqrt(lambdaPRS)*fa # Saxion mass for PRS in ADM units

In [ ]:
# INITIAL CONDITIONS
# JUST WHITE NOISE ON UNIFORM DISTRIBUTION
th = 2*pi*random.normal(size=(N,N))
phi1 = cos(th)
phi2 = sin(th)
phidot1 = zeros(shape=(N,N))
phidot2 = zeros(shape=(N,N))

# KERNEL
f1 = Laplacian_2D(phi1,dx,N) - 2*alpha*(era/t)*phidot1 - lambdaPRS*phi1*(phi1**2.0+phi2**2.0 - 1)
f2 = Laplacian_2D(phi2,dx,N) - 2*alpha*(era/t)*phidot2 - lambdaPRS*phi2*(phi1**2.0+phi2**2.0 - 1)

# EVOLUTION - VELOCITY VERLET LEAPFROG ALGORITHM IN DRIFT-KICK SCHEME
for i in range(0,tmax-1):
    # DRIFT
    phi1 = phi1 + dt*(phidot1 + 0.5*f1*dt)
    phi2 = phi2 + dt*(phidot2 + 0.5*f2*dt)
    # UPDATE
    t = t+dt
    f1_next = Laplacian_2D(phi1,dx,N) - 2*alpha*(era/t)*phidot1 - lambdaPRS*phi1*(phi1**2.0+phi2**2.0 - 1) #- lambdaPRS*T**2.0# + 10**5.0/(t**2.0) 
    f2_next = Laplacian_2D(phi2,dx,N) - 2*alpha*(era/t)*phidot2 - lambdaPRS*phi2*(phi1**2.0+phi2**2.0 - 1) #- lambdaPRS*T**2.0# + 10**5.0/(t**2.0) 
    #KICK
    phidot1 = phidot1 + 0.5*(f1 + f1_next)*dt
    phidot2 = phidot2 + 0.5*(f2 + f2_next)*dt
    # UPDATE
    f1 = 1.0*f1_next
    f2 = 1.0*f2_next
    # Physical fields
    saxion_field=sqrt(phi1**2.0+phi2**2.0)
    axion_field=arctan2(phi1,phi2)   